In [ ]:
import json
import airtable
import pandas as pd
import time

with open('settings.json') as settings_file:
    settings = json.load(settings_file)

at = airtable.AT(settings["airtable_base_key"],settings["airtable_api_key"])

In [ ]:
table = at.getTable("Pairings")
table.head()

In [ ]:
data = {
    "Teacher": "test",
    "Learner": "test",
    "Skill": "test",
    "Paired On": '2017-03-17'
}
at.pushToTable("Pairings", data)

In [ ]:
import pandas as pd

In [ ]:
records = pd.DataFrame([table["records"][j]["fields"] for j in range(len(table["records"]))]).set_index("Slack Handle")

In [ ]:
records = records[["Interests", "Skills"]]

In [ ]:
records.head()

In [ ]:
interestedLists = records["Interests"].dropna()
skilledLists = records["Skills"].dropna()
interested = pd.get_dummies(interestedLists.apply(pd.Series).stack()).sum(level=0)
skilled = pd.get_dummies(skilledLists.apply(pd.Series).stack()).sum(level=0)


In [ ]:
interested

In [ ]:
skilled

In [ ]:
learners_by_skill = {}
for column in interested.columns:
    learners_by_skill[column] = list(interested[interested[column] == 1].index)
masters_by_skill = {}
for column in skilled.columns:
    masters_by_skill[column] = list(skilled[skilled[column] == 1].index)

In [ ]:
skills_by_master = {}
for index, row in skilled.iterrows():
    skills_by_master[index] = list(skilled.transpose()[skilled.transpose()[index] == 1].index)
skills_by_learner = {}
for index, row in interested.iterrows():
    skills_by_learner[index] = list(interested.transpose()[interested.transpose()[index] == 1].index)

In [ ]:
allSkills = set(list(list(learners_by_skill.keys()) + list(masters_by_skill)))
allPeople = set(list(list(skills_by_learner.keys()) + list(skills_by_master)))

In [ ]:
allSkills, allPeople

In [ ]:
import networkx as nx
g = nx.DiGraph()
for skill, masters in masters_by_skill.items():
    for master in masters:
        learners = learners_by_skill.get(skill, [])
        for learner in learners:
            if master != learner:
                g.add_edge("T %s" % master, "L %s" % learner)
for skill, learners in learners_by_skill.items():
    for learner in learners:
        masters = masters_by_skill.get(skill, [])
        for master in masters:
            if master != learner:
                g.add_edge("T %s" % master, "L %s" % learner)

In [ ]:
maxmatch = nx.algorithms.matching.maximal_matching(g.to_undirected())

In [ ]:
maxmatch

In [ ]:
def remove_prefix(text, prefix):
    return text[text.startswith(prefix) and len(prefix):]

In [ ]:
maxmatch = nx.algorithms.matching.maximal_matching(g.to_undirected())


# Pretty-print output
# TODO: return a result in JSON or CSV format

paired_count = len(maxmatch)
print("Pairs generated: %s/%s" % (paired_count, len(allPeople)))
matched = set()
print("Pairing:")
for (a,b) in maxmatch:
    if a.startswith("T"):
        (t, l) = (a, b)
    else:
        (t, l) = (b, a)
    (tp, lp) = (remove_prefix(t, "T "), remove_prefix(l, "L "))
    matched.add(tp)
    matched.add(lp)
    ts = set(skills_by_master.get(tp, []))
    ls = set(skills_by_learner.get(lp, []))
    common = ts.intersection(ls)
    print("%s teaching %s about '%s'" % (tp, lp,"' or '".join(list(common))))
    at.create("Pairings", {
        "Teacher": tp,
        "Learner": lp,
        "Skill": str(list(common)[0]),
        "Paired On": today
    })
if paired_count < len(allPeople):
    print("People left out:")
    leftOut = allPeople.difference(matched)
    print(leftOut)
    for p in leftOut:
        at.create("Pairings", {
            "Teacher": p,
            "Learner": p,
            "Not Paired": True,
            "Paired On": today
        })

In [ ]:
maxmatch

In [ ]:
at.get("Pairings")